In [1]:
import numpy as np
import PIL
import pandas as pd
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import models, layers, Input, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, concatenate
from sklearn.metrics import f1_score

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip  /content/drive/MyDrive/HW4/HW4_data.zip

In [4]:
X_train = []
for i in range(10000):
    X_train.append(np.asarray(Image.open("HW4_data/public/train_{i}.jpg".format(i=i))))
    
df_train = pd.read_csv("HW4_data/public/train_label.csv", index_col=0)

In [5]:
X_val = []
for i in range(1000):
    X_val.append(np.asarray(Image.open("HW4_data/public/val_{i}.jpg".format(i=i))))
    
df_val = pd.read_csv("HW4_data/public/val_label.csv", index_col=0)

In [6]:
X_test = []
for i in range(1500):
    X_test.append(np.asarray(Image.open("HW4_data/private/test_{i}.jpg".format(i=i))))

df_test_demo = pd.read_csv("HW4_data/private/test_demo.csv", index_col=0)

In [7]:
Y_train = df_train['Edema'].values
X_train = np.asarray(X_train, dtype=np.float32)/255
X_val = np.asarray(X_val, dtype=np.float32)/255
X_test = np.asarray(X_test, dtype=np.float32)/255

X_train = X_train.reshape(X_train.shape[0], 128*128)
X_val = X_val.reshape(X_val.shape[0], 128*128)
X_test = X_test.reshape(X_test.shape[0], 128*128)
X_train = X_train.reshape(X_train.shape[0], 128, 128, 1)
X_val = X_val.reshape(X_val.shape[0], 128, 128, 1)
X_test = X_test.reshape(X_test.shape[0], 128, 128, 1)


In [ ]:
inputs = Input(shape = (128,128,1))
x = layers.Conv2D(filters=32, kernel_size=(4,4), activation='relu', input_shape=(128,128,1))(inputs)
x = layers.Conv2D(filters=32, kernel_size=(4,4), activation='relu')(x)
x = layers.MaxPooling2D((2,2))(x)
x = layers.Conv2D(filters=64, kernel_size=(4,4), activation='relu')(x)
x = layers.MaxPooling2D((2,2))(x)
x = layers.Dropout(0.2)(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model = Model(inputs = inputs, outputs = outputs)
model.summary()

In [9]:
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])
model.fit(X_train, df_train['Edema'], epochs = 12)

Epoch 1/12
313/313 [==============================] - 32s 72ms/step - loss: 0.4863 - accuracy: 0.7565
Epoch 2/12
313/313 [==============================] - 23s 73ms/step - loss: 0.4348 - accuracy: 0.7998
Epoch 3/12
313/313 [==============================] - 22s 72ms/step - loss: 0.4123 - accuracy: 0.8058
Epoch 4/12
313/313 [==============================] - 22s 71ms/step - loss: 0.3935 - accuracy: 0.8222
Epoch 5/12
313/313 [==============================] - 22s 71ms/step - loss: 0.3848 - accuracy: 0.8279
Epoch 6/12
313/313 [==============================] - 23s 72ms/step - loss: 0.3690 - accuracy: 0.8375
Epoch 7/12
313/313 [==============================] - 22s 72ms/step - loss: 0.3569 - accuracy: 0.8436
Epoch 8/12
313/313 [==============================] - 22s 71ms/step - loss: 0.3378 - accuracy: 0.8523
Epoch 9/12
313/313 [==============================] - 22s 71ms/step - loss: 0.3203 - accuracy: 0.8600
Epoch 10/12
313/313 [==============================] - 22s 71ms/step - loss: 0.298

In [10]:
functional_predict = model.predict(X_val)
for i in range(0,len(functional_predict)):
  if functional_predict[i] >=0.5:
    functional_predict[i] = 1
  else:
    functional_predict[i] = 0
f1_score(functional_predict, df_val['Edema'])

0.7122302158273383

In [12]:
test_predict = model.predict(X_test)
for i in range(0,len(test_predict)):
  if test_predict[i] >=0.5:
    test_predict[i] = 1
  else:
    test_predict[i] = 0
df = pd.DataFrame(test_predict)
df.to_csv('/content/drive/MyDrive/HW4/' + '108062135_basic_prediction.csv', encoding='utf-8',header = True)

In [26]:
img_inputs = Input(shape = (128,128,1))
x = layers.Conv2D(filters=32, kernel_size=(4,4), activation='relu', input_shape=(128,128,1))(img_inputs)
x = layers.Conv2D(filters=32, kernel_size=(4,4), activation='relu')(x)
x = layers.MaxPooling2D((2,2))(x)
x = layers.Conv2D(filters=64, kernel_size=(4,4), activation='relu')(x)
x = layers.MaxPooling2D((2,2))(x)
x = layers.Dropout(0.2)(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
img_dense = layers.Dense(64, activation='relu')(x)
img_model = Model(inputs = img_inputs, outputs = img_dense)
age_inputs = Input(shape = (1, ))
gender_inputs = Input(shape = (1, ))
x= layers.Dense(64, activation = 'relu')(age_inputs)
x= layers.Dense(32, activation = 'relu')(x)
y= layers.Dense(64, activation = 'relu')(gender_inputs)
y= layers.Dense(32, activation = 'relu')(y)
# z = layers.concatenate([x,y])
age_model = Model(inputs=age_inputs, outputs = x)
gender_model = Model(inputs=gender_inputs, outputs = y)
z = layers.concatenate([img_model.output, age_model.output, gender_model.output])
z = layers.Dense(32, activation='relu')(z)
z = layers.Dense(32, activation='relu')(z)
outputs = layers.Dense(1, activation='sigmoid')(z)
demographic_model = Model(inputs =[img_inputs, age_inputs, gender_inputs], outputs = outputs)
demographic_model.summary()

Model: "model_24"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_17 (InputLayer)          [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_18 (Conv2D)             (None, 125, 125, 32  544         ['input_17[0][0]']               
                                )                                                                 
                                                                                                  
 conv2d_19 (Conv2D)             (None, 122, 122, 32  16416       ['conv2d_18[0][0]']              
                                )                                                          

In [27]:
demographic_model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])
demographic_model.fit([X_train, df_train['Age'], df_train['Gender']] , df_train['Edema'], epochs = 10)

Epoch 1/10
313/313 [==============================] - 27s 79ms/step - loss: 0.4690 - accuracy: 0.7736
Epoch 2/10
313/313 [==============================] - 23s 73ms/step - loss: 0.4036 - accuracy: 0.8127
Epoch 3/10
313/313 [==============================] - 23s 72ms/step - loss: 0.3816 - accuracy: 0.8268
Epoch 4/10
313/313 [==============================] - 22s 72ms/step - loss: 0.3654 - accuracy: 0.8320
Epoch 5/10
313/313 [==============================] - 23s 73ms/step - loss: 0.3553 - accuracy: 0.8387
Epoch 6/10
313/313 [==============================] - 22s 72ms/step - loss: 0.3301 - accuracy: 0.8489
Epoch 7/10
313/313 [==============================] - 23s 72ms/step - loss: 0.3039 - accuracy: 0.8639
Epoch 8/10
313/313 [==============================] - 22s 72ms/step - loss: 0.2719 - accuracy: 0.8814
Epoch 9/10
313/313 [==============================] - 23s 73ms/step - loss: 0.2273 - accuracy: 0.8992
Epoch 10/10
313/313 [==============================] - 22s 72ms/step - loss: 0.186

In [28]:
demographical_predict = demographic_model.predict([X_val, df_val['Age'], df_val['Gender']])
for i in range(0,len(demographical_predict)):
  if demographical_predict[i] >=0.5:
    demographical_predict[i] = 1
  else:
    demographical_predict[i] = 0
f1_score(demographical_predict, df_val['Edema'])

0.7232

In [21]:
test_predict = demographic_model.predict([X_test, df_test_demo['Age'], df_test_demo['Gender']])
for i in range(0,len(test_predict)):
  if test_predict[i] >=0.5:
    test_predict[i] = 1
  else:
    test_predict[i] = 0
df = pd.DataFrame(test_predict)
df.to_csv('/content/drive/MyDrive/HW4/' + '108062135_bonus_prediction.csv', encoding='utf-8',header = True)

The advanced part, import more libraries for this part.

In [10]:
LABELS = ['Atelectasis',	'Cardiomegaly',	'Edema',	'Lung Opacity',	'No Finding',	'Pleural Effusion',	'Support Devices']


,Atelectasis,Cardiomegaly,Edema,Lung Opacity,No Finding,Pleural Effusion,Support Devices,Age,Gender
0,0,0,0,0,1,0,1,2,1
1,0,0,0,0,1,0,1,3,0
2,0,0,0,0,1,0,1,2,0
3,0,1,1,0,0,0,0,3,0
4,0,0,0,0,1,0,0,2,1
...,...,...,...,...,...,...,...,...,...
9995,0,0,0,0,1,0,0,2,0
9996,0,0,0,0,1,0,0,3,1
9997,1,0,1,1,0,1,0,3,1
9998,0,0,1,0,0,0,0,3,1


In [40]:
img_inputs = Input(shape = (128,128,1))
x = layers.Conv2D(filters=32, kernel_size=(4,4), activation='relu', input_shape=(128,128,1))(img_inputs)
x = layers.Conv2D(filters=32, kernel_size=(4,4), activation='relu')(x)
x = layers.MaxPooling2D((2,2))(x)
x = layers.Conv2D(filters=64, kernel_size=(4,4), activation='relu')(x)
x = layers.MaxPooling2D((2,2))(x)
x = layers.Dropout(0.2)(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
img_dense = layers.Dense(32, activation='relu')(x)
img_model = Model(inputs = img_inputs, outputs = img_dense)
age_inputs = Input(shape = (1, ))
gender_inputs = Input(shape = (1, ))
x= layers.Dense(64, activation = 'relu')(age_inputs)
x= layers.Dense(32, activation = 'relu')(x)
y= layers.Dense(64, activation = 'relu')(gender_inputs)
y= layers.Dense(32, activation = 'relu')(y)
# z = layers.concatenate([x,y])
age_model = Model(inputs=age_inputs, outputs = x)
gender_model = Model(inputs=gender_inputs, outputs = y)
z = layers.concatenate([img_model.output, age_model.output, gender_model.output])
z = layers.Dense(64, activation='relu')(z)
z = layers.Dense(32, activation='relu')(z)
outputs = layers.Dense(7, activation='sigmoid')(z)
advanced_model = Model(inputs =[img_inputs, age_inputs, gender_inputs], outputs = outputs)
advanced_model.summary()

Model: "model_36"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_26 (InputLayer)          [(None, 128, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_27 (Conv2D)             (None, 125, 125, 32  544         ['input_26[0][0]']               
                                )                                                                 
                                                                                                  
 conv2d_28 (Conv2D)             (None, 122, 122, 32  16416       ['conv2d_27[0][0]']              
                                )                                                          

In [30]:
dfdf = df_train.drop('Age', 1)
dfdf = dfdf.drop('Gender', 1)

In [41]:
advanced_model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(), metrics=['accuracy'])
advanced_model.fit([X_train, df_train['Age'], df_train['Gender']] , dfdf , epochs = 20)

Epoch 1/20
313/313 [==============================] - 24s 73ms/step - loss: 0.3744 - accuracy: 0.6820
Epoch 2/20
313/313 [==============================] - 23s 72ms/step - loss: 0.3417 - accuracy: 0.7009
Epoch 3/20
313/313 [==============================] - 23s 72ms/step - loss: 0.3292 - accuracy: 0.7039
Epoch 4/20
313/313 [==============================] - 23s 72ms/step - loss: 0.3160 - accuracy: 0.7045
Epoch 5/20
313/313 [==============================] - 22s 72ms/step - loss: 0.3048 - accuracy: 0.7066
Epoch 6/20
313/313 [==============================] - 22s 72ms/step - loss: 0.2949 - accuracy: 0.7103
Epoch 7/20
313/313 [==============================] - 22s 72ms/step - loss: 0.2773 - accuracy: 0.7231
Epoch 8/20
313/313 [==============================] - 22s 72ms/step - loss: 0.2634 - accuracy: 0.7335
Epoch 9/20
313/313 [==============================] - 22s 72ms/step - loss: 0.2446 - accuracy: 0.7504
Epoch 10/20
313/313 [==============================] - 22s 72ms/step - loss: 0.228

In [42]:
advanced_predict = advanced_model.predict([X_val, df_val['Age'], df_val['Gender']])
for i in range(0, len(advanced_predict)):
  for y in range(0 , 7):
    if advanced_predict[i][y] >= 0.5:
      advanced_predict[i][y] = 1
    else:
      advanced_predict[i][y] = 0
val_df = df_val.drop(['Age', 'Gender'], 1)
f1_score(advanced_predict, val_df, average = 'weighted')

0.6972720201532728

In [34]:
advanced_test_predict = advanced_model.predict([X_test, df_test_demo['Age'], df_test_demo['Gender']])
for i in range(0, len(advanced_test_predict)):
  for y in range(0 , 7):
    if advanced_test_predict[i][y] >= 0.5:
      advanced_test_predict[i][y] = 1
    else:
      advanced_test_predict[i][y] = 0
df = pd.DataFrame(advanced_test_predict)
df.to_csv('/content/drive/MyDrive/HW4/' + '108062135_advanced_prediction.csv', encoding='utf-8',header = True)